In [9]:
!pip install --upgrade openpyxl

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

#Catalogos

catalogo de productos

In [ ]:
ab=pd.read_excel("/content/drive/Shareddrives/Catalogos_SO/Prod_Holding.xlsx",engine="openpyxl")
ab.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2263 entries, 0 to 2262
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   key              2263 non-null   object
 1   Site             2263 non-null   object
 2   Holding          2263 non-null   object
 3   identificador    2150 non-null   object
 4   descripcion      2263 non-null   object
 5   CODIGO           2024 non-null   object
 6   codigo_anterior  53 non-null     object
 7   EAN              676 non-null    object
 8   Factor_Caja      2220 non-null   object
dtypes: object(9)
memory usage: 159.2+ KB


In [ ]:
# seleccion de columnas por index
ab=ab.iloc[:,[1,2,4,5]]

# filtrado sobre el catalogo
ab = ab[ab['Site'] == 'MTY']
ab = ab[ab['Holding'] == 'SU SUPER']

# selccionar columnas por index
ab= ab.iloc[:,[2,3]]

# renombrar las columnas
ab= ab.rename(columns={'identificador': "CLAVE"})
ab= ab.rename(columns={'CODIGO': "AB actual"})

In [ ]:
ab.head()

,descripcion,AB actual
1108,HIG BIG ROLL 550 H 6 PZA,AB17335636
1109,HIG BIG ROLL 550 H 8/6 PZA,AB17335636
1110,HIG BIG ROLL 600 H 6 PZA 8 PZA,AB17335636
1111,HIG ELITE COLOR 200 H 12 PZA,AB12101521
1112,HIG ELITE COLOR 200 H 32 PZA,AB11335445


catalogo de tiendas

In [ ]:
tda= pd.read_excel("/content/drive/Shareddrives/Catalogos_SO/Tda Su Super.xlsx", engine="openpyxl") # Explicitly use the openpyxl engine

tda= tda.iloc[:,[1,2]] #seleccionar columnas de interes

tda['Nombre de Tienda'] = tda['Nombre de Tienda'].str.replace('CAGSA', '')  #remplazar nombres

tda= tda.rename(columns={'Código Punto de Venta Cadena': "id"})  #renombrar una columna

tda = tda[["Nombre de Tienda", "id"]]  #reordenar las columnas

tda['Nombre de Tienda']=tda['Nombre de Tienda'].str.strip() #eliminar espacios en blanco en toda posicion

tda['Nombre de Tienda'] = tda['Nombre de Tienda'].str.lower()  #minisculas

tda.head()

/usr/local/lib/python3.11/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Nombre de Tienda,id
0,su super bodega,101
1,su super san joaquin,6
2,su super san onofre,15
3,su super 8 de julio,22
4,su super felipe angeles,25


# Semanal & Mensual

##Dataset

#### Cajas

In [11]:
cjas = pd.read_excel("/content/drive/MyDrive/Sell Out/Su Super/Semanal_Vta/Cjas/2025/01.xlsx", engine='openpyxl', engine_kwargs={'read_only': True}) # Explicitly use the openpyxl engine and read_only=True to ignore stylesheet
cjas.head()

TypeError: expected <class 'openpyxl.styles.fills.Fill'>

In [ ]:
cajas = cjas.melt(id_vars=["descripcionpdv"],
        var_name="tda",
        value_name="cajas")

cajas.head()

In [ ]:
cajas['tda'].unique()

In [ ]:
cajas = cajas[~(cajas['tda'] == 'gruposapname')]
cajas = cajas[~(cajas['tda'] == 'Total')]

In [ ]:
cajas['descripcionpdv'].unique()

In [ ]:
cajas = cajas.dropna(subset=["descripcionpdv"])

In [ ]:
#eliminar los siguientes puntos de venta
condition =(cajas['descripcionpdv'] == "TOTAL: HIGIENICO") | \
           (cajas['descripcionpdv'] == "TOTAL: PAÑAL") | \
           (cajas['descripcionpdv'] == "TOTAL: SERVILLETA") | \
           (cajas['descripcionpdv'] == "TOTAL: SERVITOALLA") | \
           (cajas['descripcionpdv'] == "TOTAL: TOALLA HUMEDA") | \
           (cajas['descripcionpdv'] == "TOTAL: VARIOS PERFUMERIA")


# Delete rows based on the condition
cajas = cajas[~condition]

valore nulos

In [ ]:
cajas.info()

In [ ]:
print("Missing values distribution: ")
print(cajas.isnull().mean())
print("")
cajas.shape

Sustituit valores negativos en 0´s

In [ ]:
cajas['cajas'] = np.where(cajas['cajas'] < 0, 0, cajas['cajas'])

#### Pesos

In [13]:
pesos = pd.read_excel("/content/drive/MyDrive/Sell Out/Su Super/Semanal_Vta/$/2025/02.xlsx",
                      engine='openpyxl') # Explicitly use the openpyxl engine
pesos.head()

TypeError: expected <class 'openpyxl.styles.fills.Fill'>

In [ ]:
pesos = pesos.melt(id_vars=["descripcionpdv"],
        var_name="tda",
        value_name="dinero")

pesos.head()

,descripcionpdv,tda,dinero
0,BULTO HIG BIG ROLL 550 H 8/6 PZA,gruposapname,HIGIENICO
1,BULTO HIG ELITE COLOR 287 H 10/4 PZA,gruposapname,HIGIENICO
2,BULTO HIG PREMIER 400 HOJAS 12/4 PZA,gruposapname,HIGIENICO
3,BULTO HIG PREMIER PLUS 600 H 10/4 PZA,gruposapname,HIGIENICO
4,BULTO HIG PREMIER RENDIPLUS 500 H 6/6 PZA,gruposapname,HIGIENICO


In [ ]:
pesos['tda'].unique()

array(['gruposapname', '01', '02', '03', '04', '05', '06', '07', '08',
       '09', 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24,
       25, 26, 27, 28, 29, 30, 31, 32, 'Total'], dtype=object)

In [ ]:
pesos = pesos[~(pesos['tda'] == 'gruposapname')]
pesos = pesos[~(pesos['tda'] == 'Total')]

In [ ]:
pesos['descripcionpdv'].unique()

array(['BULTO HIG BIG ROLL 550 H 8/6 PZA',
       'BULTO HIG ELITE COLOR 287 H 10/4 PZA',
       'BULTO HIG PREMIER 400 HOJAS 12/4 PZA',
       'BULTO HIG PREMIER PLUS 600 H 10/4 PZA',
       'BULTO HIG PREMIER RENDIPLUS 500 H 6/6 PZA',
       'HIG BIG ROLL 550 H 6 PZA', 'HIG ELITE COLOR 200 H 12 PZA',
       'HIG ELITE COLOR 200 H 32 PZA', 'HIG ELITE COLOR 200 HOJAS 4 PZA',
       'HIG ELITE COLOR 287 H 4 PZA', 'HIG ELITE PREMIUM XL 18 PZA',
       'HIG PREMIER 400 H 4 PZA', 'HIG PREMIER MAX 500 HOJAS 4 PZA',
       'HIG PREMIER PLUS 600 H 4 PZA', 'HIG PREMIER RENDI P 500 H 6 PZA',
       'BABY SEC CHICO 40 PZA', 'PAÑAL BRAZIL CHICO 40 PZA',
       'PAÑAL BRAZIL EXTRA GDE 40 PZA', 'PAÑAL BRAZIL GRANDE 40 PZA',
       'PAÑAL BRAZIL MEDIANO 40 PZA', 'PAÑAL BRAZIL PLUS MEDIANO 3 PZA',
       'SERVILLETA ELITE 180 PZA', 'SERVILLETA ELITE ULTRA 420 PZA',
       'SERVILLETA PREMIER 450 PZA', 'SERVITOALLA ELITE MEGA 1 PZA',
       'TOALLITA BABY SEC 20 PZA', 'CUBRE BOCAS ELITE 5 PZA'],
     

In [ ]:
# eliminar filas vacias a partir de una columna pivote
pesos = pesos.dropna(subset=["descripcionpdv"])

In [ ]:
#eliminar los siguientes puntos de venta
condition =(pesos['descripcionpdv'] == "TOTAL: HIGIENICO") | \
           (pesos['descripcionpdv'] == "TOTAL: PAÑAL") | \
           (pesos['descripcionpdv'] == "TOTAL: SERVILLETA") | \
           (pesos['descripcionpdv'] == "TOTAL: SERVITOALLA") | \
           (pesos['descripcionpdv'] == "TOTAL: TOALLA HUMEDA") | \
           (pesos['descripcionpdv'] == "TOTAL: VARIOS PERFUMERIA")


# Delete rows based on the condition
pesos = pesos[~condition]

valore nulos

In [ ]:
pesos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1836 entries, 0 to 2309
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   descripcionpdv  1836 non-null   object
 1   tda             1836 non-null   object
 2   dinero          1836 non-null   object
dtypes: object(3)
memory usage: 57.4+ KB


In [ ]:
print("Missing values distribution: ")
print(pesos.isnull().mean())
print("")
pesos.shape

Missing values distribution: 
descripcionpdv    0.0
tda               0.0
dinero            0.0
dtype: float64



(1836, 3)

Sustituit valores negativos en 0´s

In [ ]:
pesos['dinero'] = np.where(pesos['dinero'] < 0, 0, pesos['dinero'])

#### DF final

In [ ]:
mes = pd.merge(cajas, pesos, on=['tda','descripcionpdv'])

mes= mes.replace(r'\s+', ' ', regex=True) # espacios en blanco

mes['tda'] = mes['tda'].astype(int)

mes.head()

<ipython-input-103-8440cf92bb12>:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mes= mes.replace(r'\s+', ' ', regex=True) # espacios en blanco


,descripcionpdv,tda,cajas,dinero
0,BULTO HIG BIG ROLL 550 H 8/6 PZA,1,48.0,18000.0
1,BULTO HIG ELITE COLOR 287 H 10/4 PZA,1,4.0,818.8
2,BULTO HIG PREMIER 400 HOJAS 12/4 PZA,1,6.0,1780.2
3,BULTO HIG PREMIER PLUS 600 H 10/4 PZA,1,1.0,405.9
4,BULTO HIG PREMIER RENDIPLUS 500 H 6/6 PZA,1,58.0,15080.0


## Merged catalogos

In [ ]:
#productos
merged_mes = mes.merge(ab, left_on="descripcionpdv", right_on="descripcion", how="left")
merged_mes.head()

,descripcionpdv,tda,cajas,dinero,descripcion,AB actual
0,BULTO HIG BIG ROLL 550 H 8/6 PZA,1,48.0,18000.0,BULTO HIG BIG ROLL 550 H 8/6 PZA,AB17335636
1,BULTO HIG ELITE COLOR 287 H 10/4 PZA,1,4.0,818.8,BULTO HIG ELITE COLOR 287 H 10/4 PZA,AB12335678
2,BULTO HIG PREMIER 400 HOJAS 12/4 PZA,1,6.0,1780.2,BULTO HIG PREMIER 400 HOJAS 12/4 PZA,AB15090074
3,BULTO HIG PREMIER PLUS 600 H 10/4 PZA,1,1.0,405.9,BULTO HIG PREMIER PLUS 600 H 10/4 PZA,AB10331560
4,BULTO HIG PREMIER RENDIPLUS 500 H 6/6 PZA,1,58.0,15080.0,BULTO HIG PREMIER RENDIPLUS 500 H 6/6 PZA,AB11332659


In [ ]:
tda.head()

,Nombre de Tienda,id
0,su super bodega,101
1,su super san joaquin,6
2,su super san onofre,15
3,su super 8 de julio,22
4,su super felipe angeles,25


In [ ]:
# tiendas
merged_mes = merged_mes.merge(tda, left_on="tda", right_on="id", how="left")
merged_mes.head()

,descripcionpdv,tda,cajas,dinero,descripcion,AB actual,Nombre de Tienda,id
0,BULTO HIG BIG ROLL 550 H 8/6 PZA,1,48.0,18000.0,BULTO HIG BIG ROLL 550 H 8/6 PZA,AB17335636,su super constitucion-i,1
1,BULTO HIG ELITE COLOR 287 H 10/4 PZA,1,4.0,818.8,BULTO HIG ELITE COLOR 287 H 10/4 PZA,AB12335678,su super constitucion-i,1
2,BULTO HIG PREMIER 400 HOJAS 12/4 PZA,1,6.0,1780.2,BULTO HIG PREMIER 400 HOJAS 12/4 PZA,AB15090074,su super constitucion-i,1
3,BULTO HIG PREMIER PLUS 600 H 10/4 PZA,1,1.0,405.9,BULTO HIG PREMIER PLUS 600 H 10/4 PZA,AB10331560,su super constitucion-i,1
4,BULTO HIG PREMIER RENDIPLUS 500 H 6/6 PZA,1,58.0,15080.0,BULTO HIG PREMIER RENDIPLUS 500 H 6/6 PZA,AB11332659,su super constitucion-i,1


Observar valores faltantes en el dataset

In [ ]:
print(merged_mes.isnull().sum())

descripcionpdv      0
tda                 0
cajas               0
dinero              0
descripcion         0
AB actual           0
Nombre de Tienda    0
id                  0
dtype: int64


In [ ]:
merged_mes[merged_mes.isna().any(axis=1)]  #revisar registros con datos faltantes

#merged_mes.dropna(inplace=True) #eliminar registros con datos faltantes

,descripcionpdv,tda,cajas,dinero,descripcion,AB actual,Nombre de Tienda,id


## TD de los datasets

In [ ]:
pivot_table=pd.pivot_table(data=merged_mes,
                           index=["id", "AB actual"],
                           values=["cajas", "dinero"],
                           aggfunc=np.sum,
                           fill_value=0,)    #tabla dinamica

pivot_table.reset_index().to_excel('som.xlsx',sheet_name = 'Sheet1',index=False)

<ipython-input-110-6354a9e2209c>:1: FutureWarning: The provided callable <function sum at 0x7c5303f24a40> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_table=pd.pivot_table(data=merged_mes,
